This is based on this code: https://github.com/codekansas/keras-language-modeling/blob/master/keras_models.py

In [1]:
#%load_ext autoreload
#%autoreload 2

## Note - To Get this working:

* Install CUDA and associated libraries, setup path
* Install bleeding edge theano (from src)
* Make sure the THEANO_FLAGS are set correctly via the environment var, or via the ~/.theanorc file
* Install and compile bleeding edge Keras (from src)
* `export KERAS_BACKEND=theano`
* `export KERAS_IMAGE_DIM_ORDERING='th'`
* `sh <project_root>/shell_scipts/setup_environment.sh` to install additional dependencies
* **DO NOT SET UNROLL=True** when creating RNN's - causes max recursion issue

## Trouble-Shooting

* You may need to clean the theano cache. To do so thoroughly, run this command from the shell:
 * `theano-cache purge`

In [2]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed

import keras
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging

import datetime

Using Theano backend.


## Load and Pre-Process Essays

In [3]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [4]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [5]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-04-02 18:08:58.915382


In [6]:
from numpy.random import shuffle
shuffle(tagged_essays)

## Prepare Tags

In [7]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if (t[0].isdigit() or "->" in t) 
                     and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
vtags = set(regular_tags)
vtags.add(EMPTY_TAG)

len(unique_words)

1641

In [8]:
sorted(vtags)

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Causer:1->Result:11',
 'Causer:1->Result:13',
 'Causer:1->Result:14',
 'Causer:1->Result:2',
 'Causer:1->Result:3',
 'Causer:1->Result:4',
 'Causer:1->Result:5',
 'Causer:1->Result:50',
 'Causer:1->Result:6',
 'Causer:1->Result:7',
 'Causer:11->Result:11',
 'Causer:11->Result:12',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:11->Result:3',
 'Causer:11->Result:4',
 'Causer:11->Result:50',
 'Causer:11->Result:6',
 'Causer:12->Result:11',
 'Causer:12->Result:13',
 'Causer:12->Result:14',
 'Causer:12->Result:50',
 'Causer:12->Result:5b',
 'Causer:12->Result:7',
 'Causer:13->Result:11',
 'Causer:13->Result:12',
 'Causer:13->Result:14',
 'Causer:13->Result:4',
 'Causer:13->Result:5',
 'Causer:13->Result:50',
 'Causer:13->Result:6',
 'Causer:13->Result:7',
 'Causer:14->Result:50',
 'Causer:14->Result:6',
 'Causer:14->Result:7',
 'Causer:2->Result:1',
 'Causer:2->Result:3',
 'Causer:2->Result:50

# Transform Essays into Training Data (Word Ids)

* Computes `xs`, `ys`, `ys_bytag` and `seq_lens`
* `ys_bytag` includes **all tags** and does **not** focus only on the most common tag
* `ys` only includes the most common tag (so we can use cross entropy)
* `seq_lens` is without the start and end tags included (so we have to map back and forth to maintain mappings)
* `ys_bytag` also excludes the START and END tokens

## Get Max Sequence Length, Generate All Ids

In [9]:
ix2tag = {}
for ix, t in enumerate(vtags):
    ix2tag[ix] = t
    
generator = idGen(seed=1) # important as we zero pad sequences

maxlen = 0
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
        maxlen = max(maxlen, len(sentence) + 2)

def ids2tags(ids):
    return [generator.get_key(j) for j in ids]  

def lbls2tags(ixs):
    return [ix2tag[ix] for ix in ixs]
        
maxlen

93

In [40]:
START = "<start>"
END   = "<end>"

def get_training_data(tessays):
    # outputs
    xs = []
    ys = []
    ys_bytag = defaultdict(list)
    ys_bytag_sent = defaultdict(list)
    seq_lens = []

    # cut texts after this number of words (among top max_features most common words)
    for essay in tessays:
        for sentence in essay.sentences:
            row = []
            y_found = False
            y_seq = []
            unique_tags = set()
            for word, tags in [(START, set())] + sentence + [(END, set())]:
                id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
                row.append(id)                
                # remove unwanted tags
                tags = vtags.intersection(tags)
                unique_tags.update(tags)
                # retain all tags for evaluation (not just most common)
                # SKIP the START and END tags
                if word != START and word != END:
                    for t in (vtags - set([EMPTY_TAG])):
                        if t in tags:
                            ys_bytag[t].append(1)
                        else:
                            ys_bytag[t].append(0)

                # encode ys with all valid tags
                y_vector = []
                for t in vtags:
                    if t in tags:
                        y_vector.append(1)
                    else:
                        y_vector.append(0)
                y_seq.append(y_vector)
            
            for tag in vtags:
                if tag in unique_tags:
                    ys_bytag_sent[tag].append(1)
                else:
                    ys_bytag_sent[tag].append(0)
                
            seq_lens.append(len(row)-2)
            ys.append(y_seq)
            xs.append(row)
    
    xs = sequence.pad_sequences(xs, maxlen=maxlen)
    ys = sequence.pad_sequences(ys, maxlen=maxlen)
    assert xs.shape[0] == ys.shape[0], "Sequences should have the same number of rows"
    assert xs.shape[1] == ys.shape[1] == maxlen, "Sequences should have the same lengths"
    return xs, ys, ys_bytag, ys_bytag_sent, seq_lens

## Create Train - Test Split

In [29]:
#Helper Functions
def collapse_results(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = pred_ys[1:-1]
        for pred_tag in pred_ys:
            pred_ys_by_tag[pred_tag].append(1)
            # for all other tags, a 0
            for tag in(vtags - set([EMPTY_TAG, pred_tag])):
                pred_ys_by_tag[tag].append(0)
        if EMPTY_TAG in pred_ys_by_tag:
            del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag

In [30]:
#Helper Functions
def collapse_results_sentence_level(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = set(pred_ys[1:-1])
        for tag in vtags:
            if tag == EMPTY_TAG:
                continue
            if tag in pred_ys:
                pred_ys_by_tag[tag].append(1)
            else:
                pred_ys_by_tag[tag].append(0)
        if EMPTY_TAG in pred_ys_by_tag:
            del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag

In [31]:
def train_dev_split(lst, dev_split):
    # random shuffle
    shuffle(lst)
    num_training = int((1.0 - dev_split) * len(lst))
    return lst[:num_training], lst[num_training:]

In [41]:
%%time
# use this name for a different function later
from CrossValidation import cross_validation as cv

folds = cv(tagged_essays, CV_FOLDS)
fold2training_data = {}
fold2dev_data = {}
fold2test_data = {}

for i, (essays_TD, essays_VD) in enumerate(folds):
    # further split into train and dev test
    essays_train, essays_dev = train_dev_split(essays_TD, DEV_SPLIT)
    fold2training_data[i] = get_training_data(essays_train)
    fold2dev_data[i]      = get_training_data(essays_dev)
    # Test Data
    fold2test_data[i]     = get_training_data(essays_VD)

CPU times: user 32.8 s, sys: 3.06 s, total: 35.8 s
Wall time: 35.8 s


## Load Glove 100 Dim Embeddings

In [34]:
# see /Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/WordVectors/pickle_glove_embedding.py
# for creating pre-filtered embeddings file
import pickle, os
from numpy.linalg import norm

embeddings_file = "/Users/simon.hughes/data/word_embeddings/glove.6B/cb_dict_glove.6B.100d.txt"
# read data file
with open(embeddings_file, "rb+") as f:
    cb_emb_index = pickle.load(f)

In [35]:
missed = set()
for wd in unique_words:
    if wd not in cb_emb_index:
        missed.add(wd)
print(len(missed), len(unique_words), 100.0 * round(len(missed)/  len(unique_words),4), "%")

41 1641 2.5 %


### Construct Embedding Matrix

In [36]:
EMBEDDING_DIM = list(cb_emb_index.values())[0].shape[0]

def get_embedding_matrix(words, idgenerator, max_features, init='uniform', unit_length=False):
    embedding_dim = list(cb_emb_index.values())[0].shape[0]
    # initialize with a uniform distribution
    if init == 'uniform':
        # NOTE: the max norms for these is quite low relative to the embeddings
        embedding_matrix = np.random.uniform(low=-0.05, high=0.05,size=(max_features, embedding_dim))
    elif init =='zeros':
        embedding_matrix = np.zeros(shape=(max_features, embedding_dim), dtype=np.float32)
    elif init == 'normal':
        embedding_matrix = np.random.normal(mean, sd, size=(max_features, embedding_dim))
    else:
        raise Exception("Unknown init type")
    for word in words:
        i = idgenerator.get_id(word)
        embedding_vector = cb_emb_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    if unit_length:
        norms = np.linalg.norm(embedding_matrix, axis=1,keepdims=True)
        # remove 0 norms to prevent divide by zero
        norms[norms == 0.0] = 1.0
        embedding_matrix = embedding_matrix / norms
    return embedding_matrix

In [38]:
def score_predictions(model, xs, ys_by_tag, seq_len):
    preds = model.predict_classes(xs, batch_size=batch_size, verbose=0)   
    pred_ys_by_tag = collapse_results_sentence_level(seq_len, preds)
    class2metrics = ResultsProcessor.compute_metrics(ys_by_tag, pred_ys_by_tag)
    micro_metrics = micro_rpfa(class2metrics.values())
    return micro_metrics, pred_ys_by_tag

In [39]:
from keras.layers import Bidirectional
from datetime import datetime

def get_ts():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

def get_file_ts():
    return datetime.now().strftime('%Y%m%d_%H%M%S_%f')

embedding_size = EMBEDDING_DIM
hidden_size    = 128
out_size = len(vtags)
batch_size = 128

get_ts(), get_file_ts()

('2017-04-04 22:52:16.655171', '20170404_225216_655909')

## Train Bi-Directional LSTM With Glove Embeddings

In [73]:
max_features=len(generator.get_ids())+2 #Need plus one maybe due to masking of sequences

# merge_mode is Bi-Directional only
def evaluate_fold(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):

    if use_pretrained_embedding:
        embedding_matrix = get_embedding_matrix(unique_words, generator, max_features, init='uniform', unit_length=False)
        embedding_layer = Embedding(max_features,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=True,
                                mask_zero=True) # If false, initialize unfound words with all 0's
    else:
        embedding_layer = Embedding(max_features, embedding_size, input_length=maxlen, trainable=True, mask_zero=True)

    if bi_directional:
        rnn_layer_fact = lambda : Bidirectional(GRU(hidden_size, return_sequences=True, consume_less="cpu"), merge_mode=merge_mode)
    else:
        rnn_layer_fact = lambda : GRU(hidden_size, return_sequences=True, consume_less="cpu")
         
    model = Sequential()
    model.add(embedding_layer)
    for i in range(num_rnns):
        model.add(rnn_layer_fact())

    model.add(TimeDistributedDense(out_size))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', sample_weight_mode="temporal")

    X_train, y_train, train_ys_by_tag, train_ys_by_tag_sent, seq_len_train = fold2training_data[fold_ix]
    X_dev,   y_dev,   dev_ys_by_tag,   dev_ys_by_tag_sent,   seq_len_dev   = fold2dev_data[fold_ix]
    X_test,  y_test,  test_ys_by_tag,  test_ys_by_tag_sent,  seq_len_test  = fold2test_data[fold_ix]

    # init loop vars
    f1_scores = [-1]
    num_since_best_score = 0
    patience = 3
    best_weights = None

    for i in range(30):
        print("{ts}: Epoch={epoch}".format(ts=get_ts(), epoch=i))
        epochs = 1 # epochs per training instance
        results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=0.0, verbose=0)
        micro_metrics,_ = score_predictions(model, X_dev, dev_ys_by_tag_sent, seq_len_dev)

        print(micro_metrics)
        #print()

        f1_score = micro_metrics.f1_score
        best_f1_score = max(f1_scores)
        if f1_score <= best_f1_score:
            num_since_best_score += 1
        else: # score improved
            num_since_best_score = 0
            best_weights = model.get_weights()

        f1_scores.append(f1_score)
        if num_since_best_score >= patience:
            print("Too long since an improvement, stopping")
            break
    
    # load best weights
    model.set_weights(best_weights)
    train_micro_metrics, train_predictions_by_tag = score_predictions(model, X_train, train_ys_by_tag_sent, seq_len_train)
    test_micro_metrics,  test_predictions_by_tag  = score_predictions(model, X_test,  test_ys_by_tag_sent,  seq_len_test)
    return train_predictions_by_tag, test_predictions_by_tag, train_ys_by_tag_sent, test_ys_by_tag_sent

## Hyper Param Tuning

In [74]:
def cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    
    results = Parallel(n_jobs=1)(delayed(evaluate_fold)\
        (i, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size) for i in range(CV_FOLDS))

    cv_wd_td_ys_by_tag, cv_wd_td_predictions_by_tag = defaultdict(list), defaultdict(list)
    cv_wd_vd_ys_by_tag, cv_wd_vd_predictions_by_tag = defaultdict(list), defaultdict(list)
    for result in results:
        td_wd_predictions_by_code, vd_wd_predictions_by_code, wd_td_ys_bytag, wd_vd_ys_bytag = result
        merge_dictionaries(wd_td_ys_bytag, cv_wd_td_ys_by_tag)
        merge_dictionaries(wd_vd_ys_bytag, cv_wd_vd_ys_by_tag)
        merge_dictionaries(td_wd_predictions_by_code, cv_wd_td_predictions_by_tag)
        merge_dictionaries(vd_wd_predictions_by_code, cv_wd_vd_predictions_by_tag)
        
    SUFFIX = "_RNN"
    CB_TAGGING_TD, CB_TAGGING_VD = "CR_CB_TAGGING_TD" + SUFFIX, "CR_CB_TAGGING_VD" + SUFFIX
    parameters = dict(config)
    parameters["extractors"] = []
    parameters["min_feat_freq"] = 0

    parameters["use_pretrained_embedding"] = use_pretrained_embedding
    parameters["bi-directional"] = bi_directional
    parameters["hidden_size"] = hidden_size
    parameters["merge_mode"] = merge_mode
    parameters["num_rnns"] = num_rnns

    wd_algo = "RNN"
    wd_td_objectid = processor.persist_results(CB_TAGGING_TD, cv_wd_td_ys_by_tag, cv_wd_td_predictions_by_tag, parameters, wd_algo)
    wd_vd_objectid = processor.persist_results(CB_TAGGING_VD, cv_wd_vd_ys_by_tag, cv_wd_vd_predictions_by_tag, parameters, wd_algo)
    avg_f1 = float(processor.get_metric(CB_TAGGING_VD, wd_vd_objectid, __MICRO_F1__)["f1_score"])
    return avg_f1

In [75]:
import warnings
warnings.filterwarnings("ignore")

i = 0
for use_pretrained_embedding in [True]:
    for bi_directional in [True]:
        for num_rnns in [2]:
            for merge_mode in ["sum"]:
                for hidden_size in [256]:
                    i += 1
                    print("[{i}] Params {ts} - Embeddings={use_pretrained_embedding}, Bi-Direct={bi_directional} Num_Rnns={num_rnns} Hidden_Size={hidden_size}"\
                          .format(i=i, ts=get_ts(), use_pretrained_embedding=use_pretrained_embedding, bi_directional=bi_directional, num_rnns=num_rnns, hidden_size=hidden_size))
                    micro_f1 = cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
                    print("MicroF1={micro_f1}".format(micro_f1=micro_f1))

[1] Params 2017-04-07 22:10:05.604901 - Embeddings=True, Bi-Direct=True Num_Rnns=2 Hidden_Size=256
2017-04-07 22:10:05.904720: Epoch=0
Recall: 0.8434, Precision: 0.3858, F1: 0.5294, Accuracy: 0.9792, Codes:   881
2017-04-07 22:14:19.900991: Epoch=1
Recall: 0.8184, Precision: 0.3979, F1: 0.5355, Accuracy: 0.9803, Codes:   881
2017-04-07 22:17:35.770606: Epoch=2
Recall: 0.8354, Precision: 0.3993, F1: 0.5404, Accuracy: 0.9802, Codes:   881
2017-04-07 22:20:37.902760: Epoch=3
Recall: 0.8377, Precision: 0.4234, F1: 0.5625, Accuracy: 0.9819, Codes:   881
2017-04-07 22:23:41.399058: Epoch=4
Recall: 0.8195, Precision: 0.4252, F1: 0.5599, Accuracy: 0.9821, Codes:   881
2017-04-07 22:26:46.383272: Epoch=5
Recall: 0.8400, Precision: 0.4771, F1: 0.6086, Accuracy: 0.9850, Codes:   881
2017-04-07 22:29:54.696350: Epoch=6
Recall: 0.8331, Precision: 0.4619, F1: 0.5943, Accuracy: 0.9842, Codes:   881
2017-04-07 22:33:09.222348: Epoch=7
Recall: 0.7798, Precision: 0.4186, F1: 0.5448, Accuracy: 0.9819, Co

In [ ]:
import warnings
warnings.filterwarnings("ignore")

i = 0
for use_pretrained_embedding in [True,False]:
    for bi_directional in [True, False]:
        for num_rnns in [1,2]:
            for merge_mode in ["sum"]:
                for hidden_size in [64,128,256]:
                    i += 1
                    print("[{i}] Params {ts} - Embeddings={use_pretrained_embedding}, Bi-Direct={bi_directional} Num_Rnns={num_rnns} Hidden_Size={hidden_size}"\
                          .format(i=i, ts=get_ts(), use_pretrained_embedding=use_pretrained_embedding, bi_directional=bi_directional, num_rnns=num_rnns, hidden_size=hidden_size))
                    micro_f1 = cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
                    print("MicroF1={micro_f1}".format(micro_f1=micro_f1))

### TODO
* Use early stopping criteria
* Embeddings:
 * Don't remove low frequency words
 * Normalize all vector lengths

In [49]:
max_features=len(generator.get_ids())+2 #Need plus one maybe due to masking of sequences

# merge_mode is Bi-Directional only
fold_ix = 0
use_pretrained_embedding=True
bi_directional=True
num_rnns=1
merge_mode="sum"
hidden_size=16

if use_pretrained_embedding:
    embedding_matrix = get_embedding_matrix(unique_words, generator, max_features, init='uniform', unit_length=False)
    embedding_layer = Embedding(max_features,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True,
                            mask_zero=True) # If false, initialize unfound words with all 0's
else:
    embedding_layer = Embedding(max_features, embedding_size, input_length=maxlen, trainable=True, mask_zero=True)

if bi_directional:
    rnn_layer_fact = lambda : Bidirectional(GRU(hidden_size, return_sequences=True, consume_less="cpu"), merge_mode=merge_mode)
else:
    rnn_layer_fact = lambda : GRU(hidden_size, return_sequences=True, consume_less="cpu")

model = Sequential()
model.add(embedding_layer)
for i in range(num_rnns):
    model.add(rnn_layer_fact())

model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', sample_weight_mode="temporal")

X_train, y_train, train_ys_by_tag, train_ys_by_tag_sent, seq_len_train = fold2training_data[fold_ix]
X_dev,   y_dev,   dev_ys_by_tag,   dev_ys_by_tag_sent,   seq_len_dev   = fold2dev_data[fold_ix]
X_test,  y_test,  test_ys_by_tag,  test_ys_by_tag_sent,  seq_len_test  = fold2test_data[fold_ix]

# init loop vars
f1_scores = [-1]
num_since_best_score = 0
patience = 3
best_weights = None

for i in range(2):
    print("{ts}: Epoch={epoch}".format(ts=get_ts(), epoch=i))
    epochs = 1 # epochs per training instance
    results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=0.0, verbose=0)
    micro_metrics,_ = score_predictions(model, X_dev, dev_ys_by_tag_sent, seq_len_dev)

    print(micro_metrics)
    #print()

    f1_score = micro_metrics.f1_score
    best_f1_score = max(f1_scores)
    if f1_score <= best_f1_score:
        num_since_best_score += 1
    else: # score improved
        num_since_best_score = 0
        best_weights = model.get_weights()

    f1_scores.append(f1_score)
    if num_since_best_score >= patience:
        print("Too long since an improvement, stopping")
        break

# load best weights
model.set_weights(best_weights)
train_micro_metrics, train_predictions_by_tag = score_predictions(model, X_train, train_ys_by_tag_sent, seq_len_train)
test_micro_metrics,  test_predictions_by_tag  = score_predictions(model, X_test,  test_ys_by_tag_sent,  seq_len_test)
#return train_predictions_by_tag, test_predictions_by_tag, train_ys_by_tag, test_ys_by_tag

2017-04-05 21:41:12.188075: Epoch=0
Recall: 0.5823, Precision: 0.2043, F1: 0.3025, Accuracy: 0.9627, Codes:   881
2017-04-05 21:41:42.068373: Epoch=1
Recall: 0.5936, Precision: 0.2666, F1: 0.3679, Accuracy: 0.9716, Codes:   881


In [66]:
cv_wd_td_ys_by_tag, cv_wd_td_predictions_by_tag = defaultdict(list), defaultdict(list)
cv_wd_vd_ys_by_tag, cv_wd_vd_predictions_by_tag = defaultdict(list), defaultdict(list)
    
for fold_ix in range(5):
    print(fold_ix)
    X_train, y_train, train_ys_by_tag, train_ys_by_tag_sent, seq_len_train = fold2training_data[fold_ix]
    X_dev,   y_dev,   dev_ys_by_tag,   dev_ys_by_tag_sent,   seq_len_dev   = fold2dev_data[fold_ix]
    X_test,  y_test,  test_ys_by_tag,  test_ys_by_tag_sent,  seq_len_test  = fold2test_data[fold_ix]
    
    train_micro_metrics, train_predictions_by_tag = score_predictions(model, X_train, train_ys_by_tag_sent, seq_len_train)
    test_micro_metrics,  test_predictions_by_tag  = score_predictions(model, X_test,  test_ys_by_tag_sent,  seq_len_test)
    print(len(X_train), len(train_predictions_by_tag["50"]))
    print(len(X_test),  len(test_predictions_by_tag["50"]))
    
    td_wd_predictions_by_code, vd_wd_predictions_by_code, wd_td_ys_bytag, wd_vd_ys_bytag = train_predictions_by_tag, test_predictions_by_tag, train_ys_by_tag_sent, test_ys_by_tag_sent
    merge_dictionaries(wd_td_ys_bytag, cv_wd_td_ys_by_tag)
    merge_dictionaries(wd_vd_ys_bytag, cv_wd_vd_ys_by_tag)
    merge_dictionaries(td_wd_predictions_by_code, cv_wd_td_predictions_by_tag)
    merge_dictionaries(vd_wd_predictions_by_code, cv_wd_vd_predictions_by_tag)

0
6038 6038
1587 1587
1
5879 5879
1661 1661
2
6060 6060
1561 1561
3
5909 5909
1767 1767
4
5902 5902
1716 1716


In [68]:
parameters = dict(config)
wd_algo = "RNN"

wd_td_objectid = processor.persist_results("XXX1", cv_wd_td_ys_by_tag, cv_wd_td_predictions_by_tag, parameters, wd_algo)
wd_vd_objectid = processor.persist_results("XXX2", cv_wd_vd_ys_by_tag, cv_wd_vd_predictions_by_tag, parameters, wd_algo)